In [2]:
# 2022-08-24- 调试
from numpy import *
import numpy as np
from scipy.linalg import sqrtm
from scipy.linalg import expm, sinm, cosm
import math
import random
from qutip import *
from qutip.qobj import Qobj
from qutip.tensor import tensor
from qutip.control import *
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit
from qutip import Qobj, identity, sigmax, sigmaz,sigmay,iswap
from qutip.qobj import Qobj
from scipy.special import comb
from itertools import combinations
pi = math.pi
import numpy as np
from scipy.linalg import expm
#define  crosstalk
# to define the random circuits - 每层一个 2-qubit gate
def random_circuit_layer(pattern,totqn, evo_time_single, evo_time_two): 
        # print (pattern)
                # the operators of 1-qubit gate
        SX = Qobj(sqrtm(sigmax()))
        SY = Qobj(sqrtm(sigmay()))
        SW = Qobj(sqrtm((sigmax() + sigmay()) / (sqrt(2))))
        operator_single = [SX, SY,SW]

                # the operator of 2-qubit gate  # fsim
        theta = pi/2
        phiy = pi/6
        fsim = Qobj([[1,0,0,0],[0,cos(theta),-1j *sin(theta),0],[0,-1j * \
                    sin(theta),cos(theta),0],[0,0,0,exp(-1j*phiy)]])
        operator_two = [fsim] 

        # random circuit
        if sum(pattern==1)!=0:
                    # 1-qubit gate
                    circuit_layer_single = random.choices(operator_single, k=np.sum(pattern == 1))
                    circuit_layer = tensor(circuit_layer_single)
                    evo_time = evo_time_single
                    phi_rot = []
                    for e in circuit_layer_single:
                        if e == SX:
                            phi_rot.append(0)
                        if e == SY:
                            phi_rot.append(pi/2)
                        if e == SW:
                            phi_rot.append(pi/4)

        if  sum(pattern==1)==0:
                    evo_time = evo_time_two
                    # 2-qubit gate -正常 --- #  patternsingle = 4 的位置 pattern_all_select对应换成4
                    fsim_gate = random.choices(operator_two, k= np.sum(pattern == 3) + np.sum(pattern == 4)) # 三角矩阵
                    identity_operation = random.choices([identity(2)], k= totqn - np.sum(pattern == 4)) # 三角矩阵
                    # print (fsim_gate)
                    circuit_layer = tensor(tensor(fsim_gate),tensor(identity_operation))
        return [circuit_layer,phi_rot,evo_time]  # 好像不需要频率索引


# to calculate the evoiluiton of one layer
def evolution_matrix(totqn,evo_time_single, evo_time_two,arg,coe1,detu,\
    driveDuration,driveFrequency,driveOmegaa,freq01,freq12,coupler,pattern):
        # print (pattern)
        
        # totqn - total number of qubit
        # tg- evoluiton time---  
              # 单比特层1-qubit gate time 
              # 2-qubit gate time 分别输入
        # arg -
        # coe1 - 列向量
        # detu - detune of single qubit
        # driveDuration - ？
        # driveFrequency - driveFrequency 
        # driveOmega - 
        # freq01 - |0> -|1>
        # freq12 - |2> -|1>  # 高能级
        # coupler
        # pattern - pattern of layer
        # e.g. qubit 1,2,3
        # [[1,0,2],[2,]

        g1 = 5e6/1e9  # g1 - gc & g1
        g2 =5e6/1e9  # g2 - gc & g2
        lambda2 = lambda2=0.02
        phi= random_circuit_layer(pattern,totqn, evo_time_single, evo_time_two)[1]
        evo_time = random_circuit_layer(pattern,totqn, evo_time_single, evo_time_two)[2]

        def xpluse(amp, t, tg,sig):
            return amp*(np.exp(-(t-tg/2)**2/(2*sig**2))-np.exp(-(tg/2)**2/(2*sig**2)))

        def xplusederivator(amp, t, tg, sig):
            return (-t+tg/2)*amp*np.exp(-(t-tg/2)**2/(2*sig**2))/sig**2

        def H1_coeff(t,arg,coe1,detu,driveDuration,driveFrequency,driveOmega,phis):
            t1=t-np.trunc(t/driveDuration)*driveDuration
            xpl=xpluse(driveOmega,t1,driveDuration,driveDuration/4)
            pulx=xpl*np.cos(driveFrequency*t)
            xplder=xplusederivator(driveOmega,t1,driveDuration,driveDuration/4)
            argrel=arg-2*np.pi*detu
            puly=coe1*xplder*np.sin(driveFrequency*t)/argrel
            pul=(pulx+puly)*np.exp(2*np.pi*1j*detu*t + phis) # phis 用来区分不同方向的旋转-- x-0/y-pi/2/w-pi/4
            return pul

        def singlepart(t,qn):
            pulse=H1_coeff(t,arg[qn],coe1[qn],detu[qn],driveDuration[qn],driveFrequency[qn],driveOmegaa[qn],phi[qn])
            H0=freq01[qn]*np.array([[0,0,0],[0,1,0],[0,0,0]])+freq12[qn]*np.array([[0,0,0],[0,0,0],[0,0,1]])+\
                (pattern[qn][qn]%2)*(pulse*np.array([[0,1,0],[1,0,0],[0,0,0]])\
                    +lambda2*pulse*np.array([[0,0,0],[0,0,1],[0,1,0]]))
            s0=1
            for i in range(qn):
               s0=np.kron(s0,np.identity(3))
            for i in range(qn,qn+1):
               s0=np.kron(s0,H0)
            for i in range(qn+1,totqn):
                s0=np.kron(s0,np.identity(3))
            return s0

        def totsingle(t):
            H0=0
            for i in range(totqn):
                if pattern[i][i]==1:
                   H0=H0+singlepart(t,i)
            return H0    
        
        def couple(qn1,qn2):
            couplerfreq= coupler[qn1][qn2]
            coupling=g1*g2*(1/(couplerfreq-freq01[qn1])+1/(couplerfreq-freq01[qn2]))
            return coupling
        
        addger=np.array([[0,0,0],[1,0,0],[0,np.sqrt(2),0]])
        a2=np.transpose(addger)

        def doublepart(qn1,qn2):
            if (pattern[qn1][qn2]==3 or pattern[qn1][qn2]==4):
                s0=1
                for i in range(min(qn1,qn2)):
                    s0=np.kron(s0,np.identity(3))
                s0=np.kron(s0,addger)
                for i in range(min(qn1,qn2)+1,max(qn1,qn2)):
                    s0=np.kron(s0,np.identity(3))
                s0=np.kron(s0,a2)
                for i in range(max(qn1,qn2)+1,totqn):
                    s0=np.kron(s0,np.identity(3))
                s0=s0*couple(qn1,qn2)
            else:
                s0=0
            return s0    

        Hi=0           
        for i in range(totqn):
            for j in range(totqn):
                Hi=Hi+doublepart(i,j)
                #print (Hi)
        Hi=Hi/2

        N=500 #piece
        te= evo_time
        dt1=te/N        
        resmatrix=expm(-1j*Hi*dt1)     
        resmatrix2=expm(-1j*Hi*dt1/2)     
        s=resmatrix2  
        t0=0
        for i in range(N-1):
            t0=t0+dt1
            s=np.dot(s,expm(-1j*totsingle(t0/2)))
            s=np.dot(s,resmatrix)
        s=np.dot(s,expm(-1j*totsingle(t0/2)))
        s=np.dot(s,resmatrix)
        return s


# # to define the estimation of fidelity
# def fidelity(rho_evo,rho_ideal):
#         fid = abs(trace(Qobj(sqrtm(rho_evo)) *Qobj( array(rho_ideal)) * Qobj(sqrtm(rho_evo))))
#         return fid

# # to simulate the process xeb --全部采样
# def TOTcross_entropy_benchmark(rho_evo,rho_ideal):
#         dim = rho_ideal.shape[0]
#         prob = []
#         for i in range(dim):
#                 for j in range(dim):
#                         prob.append(np.diagonal(rho_evo)[j] * np.diagonal(rho_ideal)[i])
#                         Fidelity_XEB = sum(prob)
#         return  Fidelity_XEB

# to simulate the process xeb --部分采样
def PARcross_entropy_benchmark(rho_evo,rho_ideal):
        dim = rho_ideal.shape[0]
        prob_evo = np.diag(rho_evo)
        prob_ideal = np.diag(rho_ideal)
        nsampling = 100000
        xeb = 0
        for i in range (nsampling):
            prob_evo_sam = np.random.choice(prob_ideal,size=None,replace=True,p=prob_evo)
            xeb += prob_evo_sam
        F_XEB =xeb* dim/nsampling - 1.0
        return  F_XEB

# to define the estimation of the fidelity of the random circuits
def xeb_fidelity_simulation(totqn):

        # # parameters from DEVICE
        # totqn = DEVICE.totqubit # the number of qubits
        # nlayerd = (DEVICE.algorithm_subgraph.shape[0] - 1)  # the number of layers
        # freq01 = DEVICE.circuit_frequency_reduce[0, :] # 列向量
        # driveFrequency = DEVICE.circuit_single_drive_reduce # 列向量
        # evo_time_single= DEVICE.time_single  # 列向量 （每个门时间同 or not ）
        # evo_time_two = DEVICE.time_two  # 列向量 （每个门时间同 or not ）
        # 调试参数
        # totqn = 9 # the number of qubits
        nlayerd = 1  # the number of layers
        freq01 = np.array([4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9])
        driveFrequency = np.array([4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9,4.88e9/1e9])
        evo_time_single= 40e-8*1e9       
        evo_time_two = 12e-8*1e9       

        # parameters input-以下参数可以作为外部输入参数 or 确定不变-每个比特对应一个参数
        arg= -2.2e8/1e9 * ones(totqn) 
        coe1 = 0.05 * ones(totqn)
        detu = 0.0032 * ones(totqn)
        driveDuration = 40 * ones(totqn)
        driveOmega = 68952360.6298882/1e9 * ones(totqn)
        freq12 = freq01-2.2e8/1e9 # 高能级影响
        coupler_on = 6e9/1e9# coupler 打开时候的频率
        coupler_off = 10e9/1e9 # coupler 关闭时候的频率

        # 2-qubit 层标识-用来标记做哪个门
        pattern_two = np.array([[0,0,0,0,0,0,0,0,0],\
                                [0,2,0,0,0,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0],\
                                [0,4,0,0,2,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0],\
                                [0,0,0,0,0,0,0,0,0]])

        # 始终存在的耦合残留--输入的时候，标明是第几个qubit
        pattern_all=np.zeros([totqn,totqn])
        for e in range (totqn):
                    for f in range (totqn):
                        if np.abs(e-f)==1 or np.abs(e-f)==3:
                            pattern_all[e,f]=3
        pattern_all = np.tril(pattern_all,-1) # 只取下三角

        # 模拟线路演化
        ideal_operator_layer = []
        evo_operator_layer = []
        for i in range (nlayerd):

            if i%2==0:
                row, col = np.diag_indices_from(pattern_all)
                pattern_all[row, col] = 1
                pattern_layer = pattern_all
            if i%2!=0:
                index_two_qubit = np.argwhere(pattern_two==4 )
                pattern_all[index_two_qubit] = 4
                pattern_layer = pattern_all
            # print (pattern_layer)

            # coupler-frequency 
            coupler_layer = np.zeros([totqn,totqn])
            coupler_layer[pattern_layer == 3] =coupler_off # pattern ==3 的位置，wc = coupler_off
            coupler_layer[pattern_layer == 4] =coupler_on  # pattern ==4 的位置，wc = coupler_on
            #print (coupler)

            # ideal evolution matrix 
            circuit_layer = random_circuit_layer(pattern_layer,totqn, evo_time_single, evo_time_two)
            ideal_operator_layer.append(tensor(circuit_layer[0]))
            # print (ideal_operator_layer)

            # real evolution matrix
            evo_operator_layer.append(evolution_matrix(totqn,evo_time_single,evo_time_two,arg,coe1,detu,driveDuration,\
            driveFrequency,driveOmega,freq01,freq12,coupler_layer,pattern_layer))

        rho_evo_ideal = []
        for k in reversed(ideal_operator_layer):
             rho_evo_ideal.append(k)
        total_rho_evo_ideal = prod(rho_evo_ideal)

        rho_evo_real = []
        for j in reversed(evo_operator_layer):
             rho_evo_real.append(j)
        total_rho_evo_real = prod(rho_evo_real)

        # initial state
        rho_initial = tensor(random.choices([ket2dm(basis(2,0))],k = totqn)) 

        # iedal/real final state
        rho_evo_ideal = total_rho_evo_ideal * rho_initial  *total_rho_evo_ideal.dag()

        rho_evo_real =total_rho_evo_real * rho_initial * total_rho_evo_real.dag()
        
        # to estimate xeb of total circuit
        xeb_circuit_total = PARcross_entropy_benchmark(rho_evo_real,rho_evo_ideal)

        return xeb_circuit_total